In [1]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import tagui as t
import urllib
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [3]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows
site = getSiteByName("Today Online")
site_url = site[0][2]
site_id = site[0][0]

In [4]:
t.init(visual_automation = True, chrome_browser = True)

True

In [5]:
t.url(site_url)
t.wait(2)
t.hover('//div[@class="container footer-main"]')
t.wait(6)

True

In [6]:
number_to = t.count('(//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")])')

df_to = pd.DataFrame(index=range(0,number_to), columns = ['Sno', 'Title', 'URL', 'Summary','Img_URL'])

t.hover('//div[@class="container footer-main"]')
t.wait(2)

for n in range(1, number_to):
    title=t.read('//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][{}]//div[contains(@class, "article-listing_content")]//h2'.format(n))
    URL_o=t.read('//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][{}]//@href'.format(n))
    URL = "https://www.todayonline.com" + str(URL_o) 
    
    Img_link=t.read('//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][{}]//img/@src'.format(n))
    
    df_to.iloc[n-1, 0] = n
    df_to.iloc[n-1, 1] = title
    df_to.iloc[n-1, 2] = URL
    df_to.iloc[n-1, 4] = Img_link


In [7]:
for i in range(0, df_to.shape[0]):
    if df_to['Img_URL'][i]=="":
        df_to['Img_URL'][i]=np.nan
#str(df_to['Img_URL'][i]).startswith('http')         
df_to.dropna(subset=['Img_URL'], inplace=True, how='any')
df_to= df_to.reset_index(drop=True)
df_to['Sno'] = df_to.index

In [8]:
for n in range(0, df_to.shape[0]):
    
    t.url(df_to.URL[n])
    t.wait(4)
    t.hover('//div[@class="article-detail_subscription"]')
    t.wait(2)
    
    number_p = t.count('//div/p[not(@class)]')
    print(number_p)
    Content = ""
    
    for i in range(1, number_p-2):
        cont=t.read('//div/p[not(@class)][{}]'.format(i))
        Content = Content + "" + cont
    print(Content)   
    summaries = Summarize(df_to.Title[n], unicode(str(Content), "utf-8"))
    df_to.iloc[n-1, 3] = summaries[0]

31
SINGAPORE — From Tuesday (May 5), circuit breaker measures will be gradually eased, paving the way for certain businesses to resume operations and small groups of students to return for face-to-face lessons, a multi-ministry task force dealing with the Covid-19 outbreak said on Saturday.The measures are being eased as community transmission has decreased significantly over the past month, but National Development Minister Lawrence Wong, who co-chairs the task force, said that Singaporeans must stay disciplined and vigilant.“If there is no necessity to go out, stay at home — that must still be the mindset. Refrain from going out in groups, even if it’s with people from your own household,” he said.“We are not out of the woods… and we don't want to risk a flare up of the virus again.”When asked about whether restrictions will be eased on places of worship, Mr Wong said it will probably be “quite some time” before this happens.“We will want to refrain from any kind of close contact wit

c:\users\superhell\.conda\envs\ca1\lib\site-packages\pyteaser.py:223: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if (word not in stopWords and word in title):


15
SINGAPORE  — The suspension of the Rapid Transit System (RTS) Link project has been extended by another three months until the end of July, Singapore’s Ministry of Transport (MOT) said on Saturday (May 2).Singapore COVID-19 "circuit breaker" measures and Malaysia's movement control order have "affected the pace of our discussions", said the ministry.“Like Malaysia, we are optimistic that the discussions on the outstanding matters can be concluded within three months, using tele-conferencing and other means of communication,” it added.Malaysia’s Transport Minister Wee Ka Siong had said earlier on Saturday that bilateral discussions with Singapore on the project have been impacted by measures to stem the spread of the coronavirus on both sides of the Causeway.He added that he has discussed the status of the bilateral agreement "closely and constantly" with his Singapore counterpart Khaw Boon Wan.This is the third extension of the suspension at Malaysia's request.The RTS was meant to b

15
SINGAPORE — Residents and staff in homes serving the elderly — such as nursing homes, welfare homes, sheltered homes and adult disability homes — are receiving “priority testing” for Covid-19, the Ministry of Health (MOH) and Ministry of Social and Family Development (MSF) said on Saturday (May 2).In a statement, the two ministries added that resident-facing staff who enter and leave such homes daily will also be given lodging at designated accommodation facilities on-site or at hotels, to reduce their exposure to the community during the circuit breaker period.MOH and MSF said they are taking these steps as these homes serve highly vulnerable and frail seniors, who are at higher risk of developing serious health complications if infected with Covid-19.Broad-based testing for residents and staff in all homes serving the elderly will ensure that any Covid-19 infections in the homes are detected as early as possible for treatment, as well as to limit transmission, they said.MOH began 

In [9]:
t.close()

True

In [10]:
def insertNews(conn,news):    
    sql = ''' INSERT INTO news(site_id,create_date,title,url,summary,img_url,latest)
              VALUES(?,?,?,?,?,?,1) '''
    cur = conn.cursor()
    cur.execute(sql, news)
    return cur.lastrowid

def markNews(conn, site_id):
    cur = conn.cursor()
    cur.execute("UPDATE news set latest=0 where latest=1 and site_id=? ",(site_id,))

In [11]:
now = datetime.now()
# dd/mm/YY
dt_string = now.strftime("%d/%m/%Y")
markNews(conn, site_id)
for index, row in df_to.iterrows():
    title = row['Title'].decode('utf-8')
    url = row['URL']
    summary = row['Summary']
    img_url = row['Img_URL']
    news = (site_id,dt_string,title,url,summary,img_url)
    news_id = insertNews(conn,news)
    print("News %d has been created" % news_id)
    
conn.commit()

News 254 has been created
News 255 has been created
News 256 has been created
News 257 has been created
News 258 has been created
News 259 has been created
News 260 has been created
